A DATALOADER FOR 32 CLASSES OF GESTURE IMAGES. 24 STATIC, 8 DYNAMIC. LOAD BOTH FRAMED AND UNFRAMED, PREVENT OOM AND KERNEL CRASH. ONE-HOT ENCODING APPLIED. EXTRACT FEATURES USING A PRE-TRAINED MODEL. 

In [1]:
#import dependencies
import os
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import datetime

2023-06-21 20:31:16.706451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 20:31:16.826841: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-21 20:31:16.852687: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-21 20:31:17.399835: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
#define constants
DATA_DIR = 'fingerspelling_framed'
FRAMED_LETTERS = ['Ç','D','G','K','Ö','Ü','Y','Z']
IMG_HEIGHT = 72
IMG_WIDTH = 72
LIST_OF_LETTERS = os.listdir(DATA_DIR)

In [3]:
# create an instance of the OneHotEncoder
encoder = OneHotEncoder()

# encode the list elements
one_hot_encoded = encoder.fit_transform(np.array(LIST_OF_LETTERS).reshape(-1, 1))

# create a dictionary with the list elements as keys and their one hot encoding as values
encoding_dict = {k:v for k, v in zip(LIST_OF_LETTERS, one_hot_encoded.toarray().tolist())}

# print the dictionary
print(encoding_dict)

{'T': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'P': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'G': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'Ş': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'Y': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'Ü': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0

In [4]:
#test encodings
len(list(encoding_dict.keys()))

32

In [5]:
def resize_unframed(letter_directory):
    # Transforming images to tensors
    letter_list = []
    
    for img in os.listdir(letter_directory):
#         print(img[-3:])
        if img[-3:] == "jpg":
            #reading each image
            img = tf.io.read_file(os.path.join(letter_directory, img))
            tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)

            #resizing each image and transforming to numpy array
            tensor = tf.image.resize(tensor, [IMG_HEIGHT,IMG_WIDTH])
            # Each image 20 times
            letter_list += [[tensor]*20]
        
        
    #CONVERT PYTHON LIST OBJECT TO THE TENSOR
    
    letter_list = tf.convert_to_tensor(
                            letter_list, dtype=None, dtype_hint=None, name=None
                            )
    
#     letter_list = np.array(letter_list)
    return letter_list

In [6]:
def resize_framed(letter_directory):
    # Transforming images to tensors
    letter_list = []
    for img in os.listdir(letter_directory):
        #reading each image
        img = tf.io.read_file(os.path.join(letter_directory, img))
        tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)
        
        #resizing each image and transforming to numpy array
        tensor = tf.image.resize(tensor, [IMG_HEIGHT, IMG_WIDTH])
        # tensor += [[tensor]*20]
        letter_list.append(tensor)
        
    # Stacking tensors by 20
    x = []
    s = 0
    e = 20
    while e<=(len(letter_list)):
        temp = []

        for i in range(s, e):
            temp.append(letter_list[i])
        x.append(temp)
        s+=20
        e+=20
    
    # letter_list = np.array(letter_list)
    #convert to tensor
    letter_list = tf.convert_to_tensor(
                            x , dtype=None, dtype_hint=None, name=None
                            )
    return letter_list

In [7]:
X, Y = [], []

In [8]:
def create_data(letter):
    print(letter)
    if letter in FRAMED_LETTERS:
        X_letter = resize_framed(DATA_DIR+f"/{letter}")
        letter_data_shape =  X_letter.shape
        
    else:   
        X_letter = resize_unframed(DATA_DIR+f"/{letter}")
        letter_data_shape =  X_letter.shape
        
    
    Y_letter = [encoding_dict[letter]]*letter_data_shape[0]

    Y_letter = tf.convert_to_tensor(
                            Y_letter, dtype=None, dtype_hint=None, name=None
                         )
    print(X_letter.shape)
    return X_letter,Y_letter

In [9]:
for letter in LIST_OF_LETTERS:
    X_letter,Y_letter = create_data(letter)
    X.append(X_letter), Y.append(Y_letter)

T


2023-06-21 20:31:29.217331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 20:31:29.999882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9624 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-21 20:31:30.000397: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9560 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


(40, 20, 72, 72, 3)
P
(40, 20, 72, 72, 3)
G
(40, 20, 72, 72, 3)
Ş
(40, 20, 72, 72, 3)
Y
(40, 20, 72, 72, 3)
Ü
(40, 20, 72, 72, 3)
Q
(40, 20, 72, 72, 3)
F
(40, 20, 72, 72, 3)
U
(39, 20, 72, 72, 3)
S
(40, 20, 72, 72, 3)
A
(40, 20, 72, 72, 3)
D
(40, 20, 72, 72, 3)
H
(40, 20, 72, 72, 3)
E
(40, 20, 72, 72, 3)
İ
(40, 20, 72, 72, 3)
L
(40, 20, 72, 72, 3)
Z
(40, 20, 72, 72, 3)
N
(40, 20, 72, 72, 3)
B
(40, 20, 72, 72, 3)
O
(40, 20, 72, 72, 3)
X
(40, 20, 72, 72, 3)
J
(40, 20, 72, 72, 3)
Ö
(40, 20, 72, 72, 3)
I
(40, 20, 72, 72, 3)
R
(40, 20, 72, 72, 3)
Ç
(40, 20, 72, 72, 3)
M
(40, 20, 72, 72, 3)
V
(40, 20, 72, 72, 3)
C
(40, 20, 72, 72, 3)
Ə
(40, 20, 72, 72, 3)
K
(40, 20, 72, 72, 3)
Ğ
(40, 20, 72, 72, 3)


In [10]:
X = tf.concat(X,axis = 0)
Y = tf.concat(Y,axis = 0)

In [11]:
X.shape,Y.shape

(TensorShape([1279, 20, 72, 72, 3]), TensorShape([1279, 32]))

In [12]:
dataset = tf.data.Dataset.from_tensor_slices((X, Y))

In [13]:
dataset = dataset.shuffle(buffer_size=X.shape[0]) 

In [14]:
batch_size = 32
dataset = dataset.batch(batch_size)

In [15]:
# Testing the dataset variable
for X_batch, y_batch in dataset.take(8):
    print("X_batch shape:", X_batch.shape)
    print("y_batch shape:", y_batch.shape)

X_batch shape: (32, 20, 72, 72, 3)
y_batch shape: (32, 32)
X_batch shape: (32, 20, 72, 72, 3)
y_batch shape: (32, 32)
X_batch shape: (32, 20, 72, 72, 3)
y_batch shape: (32, 32)
X_batch shape: (32, 20, 72, 72, 3)
y_batch shape: (32, 32)
X_batch shape: (32, 20, 72, 72, 3)
y_batch shape: (32, 32)
X_batch shape: (32, 20, 72, 72, 3)
y_batch shape: (32, 32)
X_batch shape: (32, 20, 72, 72, 3)
y_batch shape: (32, 32)
X_batch shape: (32, 20, 72, 72, 3)
y_batch shape: (32, 32)
